# Causuality neural network (cMLP)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import pandas as pd
from google.colab.patches import cv2_imshow
import os
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Prepare the dataset

In [ ]:
! cp -r /content/drive/MyDrive/models/  /content/
! cp -r /content/drive/MyDrive/data/ /content/
! cp -r /content/drive/MyDrive/best_blur.pt /content/best_blur.pt
! cp -r /content/drive/MyDrive/prediction_mask/detect/  /content/
! cp /content/drive/MyDrive/SportsMOT_example/dataset/train/v_-6Os86HzwCs_c009/gt/gt.txt /content/gt.txt

In [ ]:
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.1/660.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
from models.cmlp import cMLP, cMLPSparse, train_model_ista, train_model_adam

## Define random blur and mask functino

In [ ]:
# random blur
def random_blur(img_dir, label_df, dir):
  img_list = os.listdir(img_dir, )
  for i in range(1,len(img_list)+1):
    f = f'{i:06d}.jpg'
    path = os.path.join(img_dir, f)

    img = cv2.imread(path)
    if np.random.random() > 0.7:
      label = label_df.loc[label_df.frame==i]

      obj_idx = np.random.randint(len(label))

      bb_x = int(label.values[obj_idx,2])
      bb_y = int(label.values[obj_idx,3])
      bb_w = int(label.values[obj_idx,4])
      bb_h = int(label.values[obj_idx,5])
      roi = img[bb_y:bb_y+bb_h,bb_x:bb_x+bb_w,:]

      blurred_roi = cv2.GaussianBlur(roi, (11, 11), 0)
      img[bb_y:bb_y+bb_h,bb_x:bb_x+bb_w,:] =  blurred_roi

      cv2.imwrite(os.path.join(dir, f),img)

    else:
      cv2.imwrite(os.path.join(dir, f),img)

# random mask
def random_mask(img_dir, label_df, dir):
  img_list = os.listdir(img_dir, )
  for i in range(1,len(img_list)+1):
    f = f'{i:06d}.jpg'
    path = os.path.join(img_dir, f)

    img = cv2.imread(path)
    if np.random.random() > 0.7:
      label = label_df.loc[label_df.frame==i]

      obj_idx = np.random.randint(len(label))
      bb_x = int(label.values[obj_idx,2])
      bb_y = int(label.values[obj_idx,3])
      bb_w = int(label.values[obj_idx,4])
      bb_h = int(label.values[obj_idx,5])
      img[bb_y:bb_y+bb_h,bb_x:bb_x+bb_w,:] = img[bb_y:bb_y+bb_h,bb_x:bb_x+bb_w,:] * (np.random.random()+0.5)


      cv2.imwrite(os.path.join(dir, f),img)
    else:
      cv2.imwrite(os.path.join(dir, f),img)

img_dir = 'data/custom/images'
label_df = pd.read_csv('gt.txt',header=None)
dir = 'data/blured_images/'
label_df.columns = ['frame','id','x','y','w','h','0','0','0']

In [ ]:

if not os.path.isdir(dir):
  os.mkdir(dir)
random_blur(img_dir, label_df, dir)
source = os.listdir(dir)
source = [os.path.join(dir,f) for f in source]

## Use pre-trained yolo model to detect the random-masked pictures

In [ ]:
from ultralytics import YOLO

yolo = YOLO('best_blur.pt')
#yolo.eval()
results = yolo.predict(source=source, save=True, save_txt=True,save_dir='predictions',save_conf=True)


0: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 2: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 3: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 4: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 5: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 6: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 7: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 8: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 9: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 10: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 11: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 12: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 13: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 14: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 15: 384x640 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 16: 384x640 1 0, 

In [ ]:
dir='/content/runs/detect/predict2/labels/'
all_labels = None
for i in range(1,501):
  l_f = f'{i:06d}.txt'
  label = pd.read_csv(os.path.join(dir,l_f),sep=' ',header=None)
  label.columns = ['id','x','y','w','h','prob']
  label.loc[:,'frame'] = i
  #abel = label.groupby('id',group_keys=False,).apply(lambda x: x.sort_values(by='prob', ascending=False).head(1)).head(1).reset_index(drop=True)
  label.sort_values(by='id',inplace=True)
  if all_labels is None:
    all_labels = label
  else:
    all_labels = pd.concat([all_labels,label])
all_labels.sort_values(by=['id','frame'],inplace=True)
all_labels.reset_index(drop=True,inplace=True)

In [ ]:
for i in range(1, 501):
  if not i in all_labels.loc[(all_labels['id']==0),'frame'].values:
    print(i)

In [ ]:
all_labels = all_labels.groupby(['id','frame'],as_index=False).apply(lambda x: x.sort_values(by='prob', ascending=False).head(1)).reset_index(drop=True)

In [ ]:
targets = range(10)
x = []
y = []
for key,group in all_labels.groupby('id'):


  this_x = group['x'].values
  this_y = group['y'].values

  if len(this_x) < 500:
        # Pad array with np.nan to the right
      this_x = np.pad(this_x, (0, 500 - len(this_x)), mode='constant', constant_values=this_x[-1])
      this_y = np.pad(this_y, (0, 500 - len(this_y)), mode='constant', constant_values=this_y[-1])

  x.append(this_x)
  y.append(this_y)

x = np.array(x)
y = np.array(y)


## Use predict results as cMLP inputs

In [ ]:
x_tensor = torch.from_numpy(x.T).float().unsqueeze(0)
y_tensor = torch.from_numpy(y.T).float().unsqueeze(0)

In [ ]:
x.shape

(10, 500)

In [ ]:
plt.figure()
for line in x:
  plt.plot(range(len(line)), line)

plt.savefig('multilines')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = x_tensor.to(device)
cmlp = cMLP(X.shape[-1], lag=10, hidden=[10]).to(device)

train_loss_list = train_model_ista(
    cmlp, X, lam=0.002, lam_ridge=1e-2, lr=1e-5, penalty='H', max_iter=30000,
    check_every=1000)

----------Iter = 1000----------
Loss = 0.429578
Variable usage = 100.00%
----------Iter = 2000----------
Loss = 0.377037
Variable usage = 100.00%
----------Iter = 3000----------
Loss = 0.333538
Variable usage = 100.00%
----------Iter = 4000----------
Loss = 0.297123
Variable usage = 100.00%
----------Iter = 5000----------
Loss = 0.266780
Variable usage = 100.00%
----------Iter = 6000----------
Loss = 0.241769
Variable usage = 100.00%
----------Iter = 7000----------
Loss = 0.220658
Variable usage = 100.00%
----------Iter = 8000----------
Loss = 0.202618
Variable usage = 100.00%
----------Iter = 9000----------
Loss = 0.187167
Variable usage = 100.00%
----------Iter = 10000----------
Loss = 0.173971
Variable usage = 100.00%
----------Iter = 11000----------
Loss = 0.162713
Variable usage = 100.00%
----------Iter = 12000----------
Loss = 0.153143
Variable usage = 100.00%
----------Iter = 13000----------
Loss = 0.145039
Variable usage = 100.00%
----------Iter = 14000----------
Loss = 0.13812

In [ ]:
GC_pred = cmlp.GC().cpu().numpy()


## Use True label as cMLP inputs

In [ ]:

gt_path = 'gt.txt'

df = pd.read_csv(gt_path, sep=',', header=None)

df.head()
#%%
import numpy as np
from itertools import combinations

df.sort_values(by=0, inplace=True)

unique_frames = df[0].unique()

def get_players_in_scope(df, start_frame, end_frame):
    frames_in_scope = df[df[0].between(start_frame, end_frame)]
    player_counts = frames_in_scope.groupby(1)[0].count()
    qualified_players = player_counts[player_counts == (end_frame - start_frame + 1)]
    return qualified_players.index.tolist()

longest_scope = (0, 0)
players_in_longest_scope = []

for start_frame, end_frame in combinations(unique_frames, 2):
    if end_frame - start_frame > longest_scope[1] - longest_scope[0]:
        players = get_players_in_scope(df, start_frame, end_frame)
        if len(players) >= 3:
            longest_scope = (start_frame, end_frame)
            players_in_longest_scope = players

all_possible_players = players_in_longest_scope
selected_players = np.random.choice(players_in_longest_scope, 3, replace=False) if len(players_in_longest_scope) > 3 else players_in_longest_scope

print("Longest continuous frame scope:", longest_scope)
print("All possible player IDs in the scope:", all_possible_players)
print("Selected player IDs:", selected_players)
#%%
df['central_point_x'] = df[2] + df[4] / 2
df['central_point_y'] = df[3] + df[5] / 2
#%%
df.head()
#%%
df_chosen_players = df #[df[1].isin(selected_players)]
df_chosen_players.head()
#%%
pivot_x = df_chosen_players.pivot(index=0, columns=1, values='central_point_x').fillna(0)
pivot_y = df_chosen_players.pivot(index=0, columns=1, values='central_point_y').fillna(0)
#%%
tensor_x = torch.tensor(pivot_x.values[None, :, :], dtype=torch.float32)
tensor_y = torch.tensor(pivot_y.values[None, :, :], dtype=torch.float32)

print(tensor_x.shape)
print(tensor_y.shape)
#%%
X = tensor_x.to(device)/1280
cmlp_true = cMLP(X.shape[-1], lag=10, hidden=[10]).to(device)

train_loss_list = train_model_ista(
    cmlp_true, X, lam=0.002, lam_ridge=1e-2, lr=1e-4, penalty='H', max_iter=50000,
    check_every=1000)

Longest continuous frame scope: (1, 500)
All possible player IDs in the scope: [0, 1, 2, 3, 4, 6, 7, 8, 9]
Selected player IDs: [6 3 4]
torch.Size([1, 500, 10])
torch.Size([1, 500, 10])
----------Iter = 1000----------
Loss = 0.238623
Variable usage = 100.00%
----------Iter = 2000----------
Loss = 0.161011
Variable usage = 100.00%
----------Iter = 3000----------
Loss = 0.131337
Variable usage = 100.00%
----------Iter = 4000----------
Loss = 0.118873
Variable usage = 100.00%
----------Iter = 5000----------
Loss = 0.111749
Variable usage = 100.00%
----------Iter = 6000----------
Loss = 0.106921
Variable usage = 100.00%
----------Iter = 7000----------
Loss = 0.103508
Variable usage = 100.00%
----------Iter = 8000----------
Loss = 0.101031
Variable usage = 100.00%
----------Iter = 9000----------
Loss = 0.099178
Variable usage = 100.00%
----------Iter = 10000----------
Loss = 0.097744
Variable usage = 100.00%
----------Iter = 11000----------
Loss = 0.096587
Variable usage = 100.00%
---------

In [ ]:
GC_true = cmlp_true.GC().cpu().numpy()

In [ ]:
# Make figures for GC matrix
fig, axarr = plt.subplots(1, 2, figsize=(10, 5))
axarr[0].imshow(GC_true, cmap='Blues', vmin=0, vmax=1, extent=(0, len(GC_pred), len(GC_pred), 0))
axarr[0].set_title('GC actual')
axarr[0].set_ylabel('Affected series')
axarr[0].set_xlabel('Causal series')
axarr[0].set_xticks([])
axarr[0].set_yticks([])

axarr[1].imshow(GC_pred, cmap='Blues', vmin=0, vmax=1, extent=(0, len(GC_pred), len(GC_pred), 0))
axarr[1].set_title('GC prediction')
axarr[1].set_ylabel('Affected series')
axarr[1].set_xlabel('Causal series')
axarr[1].set_xticks([])
axarr[1].set_yticks([])

# Mark disagreements
for i in range(len(GC_pred)):
    for j in range(len(GC_pred)):
        if GC_true[i, j] != GC_pred[i, j]:
            rect = plt.Rectangle((j, i-0.05), 1, 1, facecolor='none', edgecolor='red', linewidth=1)
            axarr[1].add_patch(rect)
plt.savefig('GC_comparison')
plt.show()